In [26]:
import pandas as pd  
import numpy as np 
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel
from surprise import Reader, Dataset, SVD, evaluate
from konlpy.tag import Twitter

In [27]:
original_data = pd.read_csv('./eyeShadow.csv')
type = {'건성': 0, '지성': 1, '중성': 2, '복합성': 3, '민감성': 4}
original_data['type'] = original_data['type'].map(type)

In [28]:
twitter = Twitter()
totalWord = ''
for index, data in enumerate(original_data['review']):
    words = twitter.pos(data, norm=True, stem=True)
    for word in words:
        if word[1] not in ["Punctuation", "Eomi", "Josa"]:
            totalWord += word[0] + ' '
    original_data.loc[index, 'review'] = totalWord
    totalWord = ''

,userId,name,popId,rate,review,type
0,0,매트 아이 컬러,0,4.57,진저 진짜 짱 추천 하다 코섀디 완전 자연스럽다 그냥 단독 진짜 이쁘다 웜톤 특히 추천,0
1,1,매트 아이 컬러,0,4.57,진저 평생 이 것 쓸다,0
2,2,매트 아이 컬러,0,4.57,완전 매트 다 제 좋아하다 색 이 너무 좋다,0


In [29]:
original_data.head(3)

,userId,name,popId,rate,review,type
0,0,매트 아이 컬러,0,4.57,진저 진짜 짱 추천 하다 코섀디 완전 자연스럽다 그냥 단독 진짜 이쁘다 웜톤 특히 추천,0
1,1,매트 아이 컬러,0,4.57,진저 평생 이 것 쓸다,0
2,2,매트 아이 컬러,0,4.57,완전 매트 다 제 좋아하다 색 이 너무 좋다,0


In [ ]:
id_purify_data = original_data.copy()
id_purify_data.head(3)

,userId,name,popId,rate,review,type
0,0,매트 아이 컬러,0,4.57,진저 진짜 짱 추천 하다 코섀디 완전 자연스럽다 그냥 단독 진짜 이쁘다 웜톤 특히 추천,0
1,1,매트 아이 컬러,0,4.57,진저 평생 이 것 쓸다,0
2,2,매트 아이 컬러,0,4.57,완전 매트 다 제 좋아하다 색 이 너무 좋다,0


In [7]:
member_per_type = {0: 864, 1: 849, 2: 775, 3: 855, 4: 835}
for i in range(5):
    np.random.seed(42)
    id_purify_data.loc[id_purify_data["type"] == i, "userId"] = [np.random.randint(i*200, 200*(i+1))
                                                             for j in range(0, member_per_type[i])]

In [8]:
id_purify_data.head(5)

,userId,name,popId,rate,review,type
0,102,매트 아이 컬러,0,4.57,진저 진짜 짱 추천합니다! 코섀디에도 완전 자연스럽고 그냥 단독으로도 진짜 이뻐요?...,0
1,179,매트 아이 컬러,0,4.57,진저....평생이걸로쓸거에요,0
2,92,매트 아이 컬러,0,4.57,완전 매트하고 다 제가 좋아하는 색이여서 너무 좋아요!,0
3,14,매트 아이 컬러,0,4.57,좋은 거 하나 사서 힛팬 하는 걸 목표로 프레스코 구매함. 프레스코 저렴이 많이 가...,0
4,106,매트 아이 컬러,0,4.57,한창 섀도우에 빠져 있을때 뭐 살지 고민하다가 평이 좋아서 진저 구매해서 사용해 봤...,0


In [9]:
id_purify_data = id_purify_data.sort_values('userId')

In [10]:
id_purify_data.head(10)

,userId,name,popId,rate,review,type
3453,0,제미네이트 아이즈,78,4.57,루나솔 쉐도우가 손이 많이 갑니다 .\r\n\r\n봄 여름에는 코랄코랄 많이 쓰는데...,0
2629,0,힛팬 섀도우,59,4.22,밀크티라떼 기본음영컬러임\r\n제형이 가볍고 부드러워서 발색도 잘 되고 만족스러움ㅎ...,0
2403,0,플레이 컬러 아이즈 와인파티,54,4.34,에뛰드는 기본적으로 색조화장품이 정말좋은듯??\r\n주로 버건디&브라운 조합으로 화...,0
3600,0,섀도우 커스터마이즈,82,4.30,Be286 저는 운좋게 올리브영에서\r\n한번에 샀는데 이게이렇게 구하기어려웟던건지...,0
668,0,스파클 아이섀도우,15,4.53,올스파이이뻐요ㅠㅠ펄이있어서 조금가루가날리지만\r\n예쁜눈을만들어줍니다! 화샤시하게변신,0
3996,0,호일섀도우,91,3.97,크흠..예쁘지만 팁이 너무 두꺼운 듯 합니다ㅠ,0
91,1,피스 매칭 섀도우 매트,2,4.61,세일할 때 사면 가격도 굉장히 싸고 가루날림이 적고 발색력이 좋다!\r\n드라이 로...,0
1071,1,리미티드 아이 아티스트 팔레트,24,4.88,가을 웜톤인데 로라메르시에 아이섀도우는 워낙 유명해서 이번엔 첫 로라 아이섀도우 구...,0
2665,1,매그니피센트 메탈 글리터&글로우 리퀴드 아이섀도우 듀오 크롬 쉐이드,60,4.27,진짜 케이스도이쁘고 반짝이있는거눈밑에바르면 막 여기저기붙기도하는데 이건접착력?이좋아...,0
1374,1,프로 롱웨어 페인트 팟,31,4.41,초반에는 엄천 잘 썼었어요 부드럽게 발리고 발색도 은은하니 예쁘고 근데 빨리 굳어버...,0


In [11]:
def reviewData(type=0):
    review_data = ['' for i in range(96)]
    review_type_popid_data = pd.DataFrame(id_purify_data[["popId", "type", "review"]])
    review_type_popid_data = review_type_popid_data.reset_index(drop=True)
    review_type_popid_data = review_type_popid_data.loc[review_type_popid_data["type"]==type,:]
    for index, row in review_type_popid_data.iterrows():
        review_data[row["popId"]] += row["review"]
    return review_data

In [12]:
review_data= reviewData(0)

In [13]:
len(review_data)

96

In [14]:
id_to_name = original_data[["popId", "name"]].drop_duplicates()
id_to_name = id_to_name.reset_index(drop=True)
id_to_name = id_to_name.drop(columns="popId", axis=1)
id_to_name

,name
0,매트 아이 컬러
1,아이섀도우
2,피스 매칭 섀도우 매트
3,모던 섀도우 이탈프리즘 - 쉬머
4,새틴 아이컬러
5,피스매칭 12구 섀도우 팔레트
6,문더스트 아이섀도우 파 아 포피에르
7,네이키드 히트
8,매그니피센트 메탈 글리터 앤 글로우 리퀴드 아이섀도우
9,스위트 피치 아이 섀도우 컬렉션


In [15]:
name_to_id = original_data[["popId", "name"]].drop_duplicates()
name_to_id = name_to_id.set_index("name", drop=True )

In [16]:
review_data = pd.Series(review_data)
tf = TfidfVectorizer(analyzer='word', min_df=2, stop_words=['\r', '\n'], sublinear_tf=True)
tf_matrix = tf.fit_transform(review_data)
cosine_sim = linear_kernel(tf_matrix, tf_matrix)

In [17]:
def making_evaluate_data(type = 0):
    evaluate_data = id_purify_data[id_purify_data["type"]==type]
    evaluate_data = evaluate_data.drop(columns=['name', 'review', 'type'], axis=1)
    print(evaluate_data)
    reader = Reader()
    evaluate_data = Dataset.load_from_df(evaluate_data, reader)
    return evaluate_data

In [18]:
evaluate_data = making_evaluate_data(0)

      userId  popId  rate
3453       0     78  4.57
2629       0     59  4.22
2403       0     54  4.34
3600       0     82  4.30
668        0     15  4.53
3996       0     91  3.97
91         1      2  4.61
1071       1     24  4.88
2665       1     60  4.27
1374       1     31  4.41
357        1      8  4.47
3950       2     90  4.15
3604       2     82  4.30
4085       2     93  4.29
1198       2     27  4.33
896        2     20  4.39
4035       3     92  4.55
2631       3     59  4.22
361        3      8  4.47
1730       3     39  4.08
669        4     15  4.53
3815       4     87  4.35
3276       4     74  4.21
576        4     13  4.46
2223       4     50  4.44
1775       5     40  4.18
2226       5     50  4.44
3945       5     90  4.15
3234       6     73  4.30
1332       6     30  4.42
...      ...    ...   ...
4175     191     95  4.43
4037     192     92  4.55
628      192     14  5.00
4124     192     94  4.16
1547     193     35  4.29
3810     193     87  4.35
1949     193

In [19]:
evaluate_data.split(5)
svd = SVD()
evaluate(svd, evaluate_data, measures=['RMSE', 'MAE'])

C:\Users\LeeSangHoon\Anaconda3\envs\tensorflow\lib\site-packages\surprise\evaluate.py:66: UserWarning: The evaluate() method is deprecated. Please use model_selection.cross_validate() instead.
  'model_selection.cross_validate() instead.', UserWarning)
C:\Users\LeeSangHoon\Anaconda3\envs\tensorflow\lib\site-packages\surprise\dataset.py:193: UserWarning: Using data.split() or using load_from_folds() without using a CV iterator is now deprecated. 
  UserWarning)


Evaluating RMSE, MAE of algorithm SVD.

------------
Fold 1


RMSE: 0.1456
MAE:  0.1138
------------
Fold 2
RMSE: 0.1318
MAE:  0.1076
------------
Fold 3


RMSE: 0.1509
MAE:  0.1157
------------
Fold 4


RMSE: 0.1434
MAE:  0.1161
------------
Fold 5
RMSE: 0.1453
MAE:  0.1118
------------
------------
Mean RMSE: 0.1434
Mean MAE : 0.1130
------------
------------


CaseInsensitiveDefaultDict(list,
                           {'rmse': [0.14559505497135847,
                             0.13178146356917553,
                             0.15087884371319718,
                             0.1434095456801219,
                             0.14533684056966803],
                            'mae': [0.11379915114748992,
                             0.10757760250840351,
                             0.11569783061258948,
                             0.11605280502882057,
                             0.11180701230139657]})

In [20]:
from sklearn.externals import joblib

trainset = evaluate_data.build_full_trainset()
svd.fit(trainset)
joblib.dump(svd, 'eyeShadow.pkl')

['eyeShadow.pkl']

In [21]:
def making_predict_data(cosmetic_id):
    predict_data = original_data[['popId', 'name']]
    predict_data = predict_data.drop_duplicates()
    return predict_data.iloc[cosmetic_id]

In [22]:
def get_recommaned_cosmetic(userId, cosmeticName='매트 아이 컬러'):
    cosmetic_id = name_to_id.loc[cosmeticName,:]
    sim_scores = list(enumerate(cosine_sim[int(cosmetic_id)]))
    sim_scores = sorted(sim_scores, key=lambda x:x[1], reverse=True)
    sim_scores = sim_scores[1:26]
    cosmetic_id = [i[0] for i in sim_scores]
    prediction = making_predict_data(cosmetic_id)
    prediction['est'] = prediction['popId'].apply(lambda x: svd.predict(userId, x).est)
    prediction = prediction.sort_values('est', ascending=False)
    return prediction

In [24]:
get_recommaned_cosmetic(15, '네이키드 히트')

,popId,name,est
1682,38,꾸뛰르 섀도우,4.429940
3229,73,글로시 리치 아이즈,4.393096
3103,70,아이섀도,4.391241
2399,54,플레이 컬러 아이즈 와인파티,4.374487
4169,95,컬러 마스터 아이 팔레트,4.361302
1949,44,마이 아이섀도우 - 매트,4.354240
3312,75,샘물 싱글 섀도우(매트),4.350664
1637,37,아이섀도우 새틴,4.309936
1152,26,아티스트 섀도우,4.294562
2624,59,힛팬 섀도우,4.257228
